## Latend Dirichlet Allocation (LDA) - News Articles dataset

Musat Bianca-Stefania

407 Artificial Intelligence

In [68]:
# import all necessary packages
import os
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import download
download('stopwords')
download('wordnet')
download('punkt')

import numpy as np
from collections import Counter
!pip install pymc
import pymc as pm
import matplotlib.pyplot as plt
from matplotlib import rcParams
from pymc3 import traceplot
from math import log, exp

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Dataset

The dataset used for this project is called BBC News Summary and can be downloaded from the following link: https://www.kaggle.com/pariza/bbc-news-summary. The reason I chose this dataset is that it consists of a large number of news documents that are divided into 5 subcategories.
 
 
In the dataset archive, there is a folder named "News Articles" that contains 5 subfolders for 5 news categories. Each category consists of several hundred documents.
 
For this project, I have chosen only 3 categories (tech, sport, and politics) and from each category, I extracted the first 150 words of the first 10 documents.

In [69]:
NO_FILES = 10  # number of files/documents to be chosen from each category
NO_WORDS = 150  # number of words to be extracted from each document
CAT_NAMES = ["politics", "sport", "tech"]  # names of the categories we want to use (business, entertainment, politics, sport, tech)

Reading data from dataset. We will read the first NO_FILES documents from each category (from one category we read NO_FILES + 1 in order to use the +1 document for the final task). The categories used are the ones defined in CAT_NAMES.

In [70]:
# read data from data set
dataset = []  # dataset used for the model (before preprocessing)
labels = []  # list of labels (categories) for each document in order
categories = []  # list of all distinct categories
new_file = ""  # file used for the last task

for cat_name in os.listdir("./News Articles"):
    if cat_name[0] == '.':  # removes hidden files
        continue
    if cat_name not in CAT_NAMES:  # do no read articles from other topics then the ones specified in CAT_NAMES
        continue

    print("Category: ", cat_name)
    dir_path = "./News Articles/" + cat_name
    categories.append(cat_name)
    print(os.listdir(dir_path))  # print all files in current directory

    no_files = NO_FILES
    if cat_name == CAT_NAMES[0]:  # the file used for the last task will be chosen from the first category in the list
        no_files += 1

    for filename in os.listdir(dir_path):
        if filename.startswith(".") or no_files == 0:  # removes hidden files
            continue
        else:
            no_files -= 1
            file_path = dir_path + "/" + filename
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read().strip()
            if no_files == NO_FILES:  # if this is the file used for the last task, store separately
                new_file = content
            else:
                dataset.append(content)
                labels.append(cat_name)

Category:  sport
['009.txt', '008.txt', '021.txt', '010.txt', '006.txt', '020.txt', '016.txt', '014.txt', '015.txt', '013.txt', '012.txt', '002.txt', '018.txt', '003.txt', '004.txt', '005.txt', '011.txt', '019.txt', '007.txt', '017.txt', '001.txt']
Category:  politics
['009.txt', '008.txt', '010.txt', '006.txt', '016.txt', '014.txt', '015.txt', '013.txt', '012.txt', '002.txt', '018.txt', '003.txt', '004.txt', '005.txt', '011.txt', '007.txt', '017.txt', '001.txt']
Category:  tech
['009.txt', '008.txt', '021.txt', '010.txt', '006.txt', '020.txt', '016.txt', '014.txt', '015.txt', '013.txt', '012.txt', '002.txt', '018.txt', '003.txt', '004.txt', '005.txt', '011.txt', '019.txt', '007.txt', '017.txt', '001.txt']


In [71]:
print("Dataset length: ", len(dataset))
print("Dataset content: ", dataset)
print("Labels content: ", labels)
print("Categories content: ", categories)
print("\nFile used for last task:\n\n", new_file)

Dataset length:  30
Dataset content:  ['Off-colour Gardener storms to win\n\nBritain\'s Jason Gardener shook off an upset stomach to win the 60m at Sunday\'s Leipzig International meeting.\n\nGardener clocked 6.56 seconds to equal the meeting record and finished well ahead of Germany\'s Marc Blume, who crossed the line in 6.67 secs. The world indoor champion said: "I got to the airport and my stomach was upset and I was vomiting. I almost went home. "I felt a little better Sunday morning but decided I\'d only run in the main race. Then everything went perfectly." Gardener, part of the Great Britain 4x100m quartet that won gold at the Athens Olympics, will now turn his attention to next weekend\'s Norwich Union European Indoor trials in Sheffield.\n\n"Given I am still off-colour I know there is plenty more in the tank and I expect to get faster in the next few weeks," he said. "It\'s just a case of chipping away as I have done in previous years and the results will come." Scotland\'s Ia

## Data preprocessing

In order to preprocess the data, I have tokenized the sentences from each documents into words using nltk. Then, I have chosen the first NO_WORDS from each document and I have removed the stop words, any word shorter then 2 characters and I have lemmatized the words so that only the root of each word was stored.

The transformed dataset was stored in the same variable

In [72]:
stop_words = list(stopwords.words('english'))  # get list of stop words

def process_text(dataset, use_stop_words = True, use_lematization = True):
    lemmatizer = WordNetLemmatizer()
    for indx, doc in enumerate(dataset):  # for all documents in the dataset
      list_of_words = word_tokenize(doc)  # tokenze each document into words
      dataset[indx] = list_of_words[0:NO_WORDS]  # keep the first NO_WORDS words
      if use_stop_words:  # remove stop words and words shorter then 2 characters (used mostly for punctuation)
        dataset[indx] = [word.lower() for word in dataset[indx] if word.lower() not in stop_words and len(word) > 2]
      if use_lematization:  # lemmatize words
        dataset[indx] = [lemmatizer.lemmatize(word) for word in dataset[indx]]

In [73]:
process_text(dataset)

In [74]:
print(dataset)

[['off-colour', 'gardener', 'storm', 'win', 'britain', 'jason', 'gardener', 'shook', 'upset', 'stomach', 'win', '60m', 'sunday', 'leipzig', 'international', 'meeting', 'gardener', 'clocked', '6.56', 'second', 'equal', 'meeting', 'record', 'finished', 'well', 'ahead', 'germany', 'marc', 'blume', 'crossed', 'line', '6.67', 'sec', 'world', 'indoor', 'champion', 'said', 'got', 'airport', 'stomach', 'upset', 'vomiting', 'almost', 'went', 'home', 'felt', 'little', 'better', 'sunday', 'morning', 'decided', 'run', 'main', 'race', 'everything', 'went', 'perfectly', 'gardener', 'part', 'great', 'britain', '4x100m', 'quartet', 'gold', 'athens', 'olympics', 'turn', 'attention', 'next', 'weekend', 'norwich', 'union', 'european', 'indoor', 'trial', 'sheffield', 'given', 'still', 'off-colour', 'know', 'plenty'], ['hansen', "'delays", 'return', "2006'", 'british', 'triple', 'jumper', 'ashia', 'hansen', 'ruled', 'comeback', 'year', 'setback', 'recovery', 'bad', 'knee', 'injury', 'according', 'report', 

In [75]:
new_f = list([new_file])
process_text(new_f)  # preprocess the file that will be used for the  final task

In [76]:
print(new_f)

[['campbell', 'e-mail', 'row', "'silly", "fuss'", 'ex-no', 'medium', 'chief', 'alastair', 'campbell', 'centre', 'new', 'political', 'row', 'e-mail', 'containing', 'four-letter', 'outburst', 'aimed', 'bbc', 'journalist', 'campbell', 'sent', 'missive', 'mistake', 'bbc2', 'newsnight', 'sought', 'question', 'role', 'labour', 'controversial', 'poster', 'campaign', 'later', 'contacted', 'show', 'saying', 'original', 'e-mail', 'sent', 'error', 'silly', 'fuss', 'campbell', 'recently', 're-joined', 'labour', 'election', 'campaign', 'e-mail', 'revealed', 'day', 'peter', 'mandelson', 'former', 'labour', 'minister', 'european', 'commissioner', 'warned', 'bbc', 'steer', 'away', 'demonising', 'campbell', 'campbell', 'messaged', 'newsnight', 'programme', 'investigated', 'claim', 'labour', 'advertising', 'agency', 'tbwa', 'blaming', 'controversy', 'campaign', 'poster']]


## Transform string data from dataset into numbers so that it can be used by LDA

We will define a vocabulary which is a dictionary that maps each word to a number. We also define an index_to_word dictionary which allows us to easily trace back each word from the associated number.

In [78]:
def make_vocab(dataset):
    vocabulary = {}  # dict {word : index}
    index_to_word = {}  # dict {index : word}
    index = 0

    for item in dataset:
        for word in item:
            if word not in vocabulary:
                vocabulary[word] = index
                index_to_word[index] = word
                index += 1
    return vocabulary, index_to_word

In [79]:
vocab, index_to_word = make_vocab(dataset)
print("Vocabulary length: ", len(vocab))

Vocabulary length:  1276


In [80]:
def process_dataset(dataset):  # transform dataset into number representation
    for i, item in enumerate(dataset):
        for j, word in enumerate(item):
            if word in vocab:
                dataset[i][j] = vocab[word]
            else:  # if word not in vocabulary, assign new index
                dataset[i][j] = len(vocab)

In [81]:
process_dataset(dataset)
process_dataset(new_f)

In [82]:
print("Dataset: ", dataset)
print("New file: ", new_f)

Dataset:  [[0, 1, 2, 3, 4, 5, 1, 6, 7, 8, 3, 9, 10, 11, 12, 13, 1, 14, 15, 16, 17, 13, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 8, 7, 35, 36, 37, 38, 39, 40, 41, 10, 42, 43, 44, 45, 46, 47, 37, 48, 1, 49, 50, 4, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 30, 63, 64, 65, 66, 0, 67, 68], [69, 70, 71, 72, 73, 74, 75, 76, 69, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 69, 87, 62, 31, 88, 89, 62, 90, 91, 92, 93, 94, 95, 96, 71, 97, 98, 84, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 74, 75, 103, 32, 69, 110, 96, 71, 111, 112, 113, 108, 114, 115, 116, 117, 118, 119, 98, 120, 121, 122], [123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 123, 136, 137, 79, 128, 127, 138, 126, 123, 139, 56, 140, 141, 142, 143, 144, 145, 146, 12, 136, 32, 147, 148, 149, 150, 151, 152, 153, 154, 155, 29, 156, 19, 16, 157, 158, 159, 160, 132, 46, 161, 162, 163, 164, 165, 133, 166, 167, 79, 126, 168, 131, 29, 169, 170, 171, 172, 158, 79, 132, 118, 173, 174, 175,

# LDA

In the next section I have implemented and described the LDA model.

LDA is a probabilistic topic model in which each document in the dataset exhibits multiple topics in different proportions, each topic being a distribution over the predefined vocabulary.

LDA assumes that the documents are created via a generative process that looks like this: we first choose a distribution over topics for each document, and for each word in that document we choose a random topic from that distribution and then we choose a word from the correct distribution over the vocabulary. As stated before, this generative process will lead to documents belonging to multiple topics in different proportions.

LDA reverse engeneer this process. In order to do that, it has to draw a topic for each word in each document, and then to draw the real word from the correct distribution (which is the distribution associated with the topic). The distribution that we use to draw the topic/word is Categorical distribution. The parameters used for these 2 distributions are going to be drawn from a Dirichlet distribution, as this is the conjugate prior for the Categorical.

z[m,n] = Categorical(theta[m])  # drawing a topic for each word (each word n in each document m)

w[m,n] = Categorical(phi[z[m,n]])   # drawing the physical word (each word n in each document m)

theta[m] = Dirichlet(alpha)  # topic distribution for document m (it will be K dimensional, as there ar K topics in total)

phi[k] = Dirichlet(beta)  # word distribution for topic k (it will be V dimentional, as there are V possible words in the dataset)

In [83]:
K = len(categories)  # number of categories/topics
M = len(dataset)  # number of documents in the dataset
V = len(vocab)  # vocabulary length

In [84]:
def lda(dataset, alpha=np.ones(K), beta=np.ones(V)):
    Nm = []  # number of words in each document
    for d in dataset:
      Nm.append(len(d))

    # draw word distribution for each topic ( dimentional, as there are V possible words in the dataset)
    phi1 = [pm.Dirichlet("phi1_%i" % i, theta=beta) for i in range(K)]
    phi = [pm.CompletedDirichlet("phi_%i" % i, phi1[i]) for i in range(K)]
    phi = pm.Container(phi)

    # draw topic distribution for each document (K dimensional, as there ar K topics in total)
    theta1 = [pm.Dirichlet("theta1_%i" % i, theta=alpha) for i in range(M)]
    theta = [pm.CompletedDirichlet("theta_%i" % i, theta1[i]) for i in range(M)]
    theta = pm.Container(theta)

    rand_topic = []  # randomly choose a topic for each word in each document
    for i in range(M):
      rand_topic.append(np.random.randint(K, size=Nm[i]))

    # draw a topic for each word in each document
    Z = [pm.Categorical("Z1_%i" % i, p=theta[i], size=Nm[i], value=rand_topic[i]) for i in range(M)]
    Z = pm.Container(Z)

    # draw the word itself from the word distribution
    W = [pm.Categorical("w_%d_%i" % (d,i), p = pm.Lambda("z_%d_%i" % (d,i), lambda z=Z[d][i], phi=phi : phi[z]), value=dataset[d][i], observed=True) for d in range(M) for i in range(Nm[d])]
    W = pm.Container(W)

    # create LDA model
    model = pm.Model([theta, phi, Z, W, phi1, theta1])

    # sample values
    mcmc = pm.MCMC(model)
    trace = mcmc.sample(iter=20000, burn=1000)
  
    print("\nThe topic of each word in each document:\n")
    for m in range(M):  
      tr = mcmc.trace('Z1_%i' % m)[20000 - 1000 - 1]
      print(tr)

    print("\nThe mean topic of each word in each document:\n")
    for m in range(M):  
      tr = mcmc.trace('Z1_%i' % m)[-1000:-1].mean(axis=0)
      print(tr)
    
    # for m in range(M):  
    #   traceplot(mcmc.trace('Z1_%i'% m )[-1000:-1].mean(axis=0))

    # print("\nThe distribution of words for each topic:\n")  ##################### ia primele cele mai frecvente cuvinte
    # for k in range(K):
    #   print("Topic ", k)
    #   for i, j in enumerate(mcmc.trace('phi_%i' % k)[-1000:-1].mean(axis=0)[0]):
    #       print("   ", index_to_word[i], " ", j)
    #   print()

    print("\nTopic distribution for each document:\n")
    for m in range(M):
      print("Document ", m)
      for i, j in enumerate(mcmc.trace('theta_%i' % m)[-1000:-1].mean(axis=0)[0]):
          print("Topic ", i, " ", j)
      print()

    return (theta, phi, Z, W)

## Choosing alpha and beta

Alpha and beta parameters are fixed and it is therefore important to choose them carefully. They affect the sparcity of the model in the following way:

- alpha controls the document topic density, so we want a higher alpha when we have more topics
- beta controls word topic density, so we want a higher beta when the number of words in vocabulary is high

In our case, we have a small number of topics, so we would rather choose a small alpha, but a moderate number of words in the dictionary, so intuitively, beta should be not too high, not too low. Of course, the best way of choosing alpha and beta is still to empirically test for a number of values.

## Choosing a way to measure model success

As a measure of success, I have chosen to implement the accuracy per topic. So, I am choosing the correct topic for a document as being the most frequent topic among those documents that should belong to the same topic. Then I compute how many words have been correctly assign to that topic.

In [49]:
# compute the accuracy for each topic
def topic_acc(Z_matrix):
    indx = 1
    counters = []
    maxims = []
    lenghts = []
    print()
    for lst in Z_matrix:
        counters.append(Counter(lst))  # for each topic, we count how many words have been assign to it
        maxims.append(max(lst, key=lst.count))  # for each document, we get the most common topic
        lenghts.append(len(lst))  # for each document, we get the number of words in it
        if indx % NO_FILES == 0:
            acc = 0
            tot = 0
            i = 0
            for c in counters:
                acc += c[max(maxims, key=maxims.count)] / lenghts[i]  # compute accuracy as correctly assigned words / total number of words
                i += 1
            
            print("Topic: ", max(maxims, key=maxims.count))
            print("Topic accuracy ", acc / NO_FILES)
            print("Most frequent topic in each document belonging to the real topic: ", maxims)
            print()
            counters = []
            maxims = []
            lenghts = []
        indx += 1

In [ ]:
(theta, phi, Z, W) = lda(dataset)

/usr/local/lib/python3.7/dist-packages/pymc/MCMC.py:81: UserWarning: Instantiating a Model object directly is deprecated. We recommend passing variables directly to the Model subclass.
  warnings.warn(message)


 [-----------------100%-----------------] 20000 of 20000 complete in 2481.9 sec
The topic of each word in each document:

[1 0 0 2 2 1 0 1 1 2 0 1 1 1 1 2 0 0 1 1 1 0 0 1 0 0 1 0 2 0 1 2 0 2 0 1 2
 0 0 2 0 0 0 0 1 0 1 2 1 2 1 0 0 2 2 0 2 1 0 0 2 1 0 2 1 2 0 0 0 2 0 1 1 0
 0 0 2 2 1 2 1]
[2 0 0 2 2 2 2 1 0 2 2 2 0 0 0 2 0 2 0 0 2 1 2 2 0 1 2 2 2 2 0 2 2 2 0 0 1
 2 2 1 0 2 1 2 1 0 2 2 2 2 2 2 0 2 1 2 0 0 2 2 0 0 2 1 0 2 1 0 0 0]
[2 2 2 2 0 2 2 0 1 2 1 1 1 2 2 2 2 1 2 0 1 2 2 2 1 2 2 1 1 0 1 2 2 2 2 0 2
 1 0 2 2 0 0 2 2 1 1 0 2 2 0 2 2 2 1 0 1 0 1 0 0 1 2 0 2 0 1 0 1 0 2 0 1 2
 2 0 0 0 1 2 2 0 0 2 1 2]
[0 2 2 2 2 2 1 2 0 2 0 1 1 2 1 1 2 2 2 2 2 2 1 2 1 2 1 1 1 2 2 2 2 2 2 0 1
 0 1 1 1 2 1 2 2 2 0 1 2 1 2 1 1 2 2 2 0 1 2 2 2 1 2 2 1 2 2 2 2 2 2 2 2 2
 2 2 1 1 2]
[0 0 0 0 0 0 0 2 0 0 2 1 1 0 1 0 1 1 0 0 2 1 0 0 2 0 2 0 1 1 0 1 2 2 0 2 0
 2 2 0 1 2 2 1 1 2 2 2 0 0 1 0 1 2 2 0 0 1 2 0 2 0 2 2 0 0 0 1 1 2 1 1 2 0
 1 1]
[0 0 2 2 2 2 1 1 0 0 2 1 0 0 2 1 2 2 0 2 1 0 1 1 2 2 2 1 1 1 0 0 0 2 0 0 1


In [ ]:
Z_matrix = []
for i in range(len(Z)):
        Z_matrix.append(list(Z[i].value))
print(Z_matrix)

topic_acc(Z_matrix)

[[1, 0, 0, 2, 2, 1, 0, 1, 1, 2, 0, 1, 1, 1, 1, 2, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 2, 0, 1, 2, 0, 2, 0, 1, 2, 0, 0, 2, 0, 0, 0, 0, 1, 0, 1, 2, 1, 2, 1, 0, 0, 2, 2, 0, 2, 1, 0, 0, 2, 1, 0, 2, 1, 2, 0, 0, 0, 2, 0, 1, 1, 0, 0, 0, 2, 2, 1, 2, 1], [2, 0, 0, 2, 2, 2, 2, 1, 0, 2, 2, 2, 0, 0, 0, 2, 0, 2, 0, 0, 2, 1, 2, 2, 0, 1, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 1, 2, 2, 1, 0, 2, 1, 2, 1, 0, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 0, 0, 2, 2, 0, 0, 2, 1, 0, 2, 1, 0, 0, 0], [2, 2, 2, 2, 0, 2, 2, 0, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 2, 0, 1, 2, 2, 2, 1, 2, 2, 1, 1, 0, 1, 2, 2, 2, 2, 0, 2, 1, 0, 2, 2, 0, 0, 2, 2, 1, 1, 0, 2, 2, 0, 2, 2, 2, 1, 0, 1, 0, 1, 0, 0, 1, 2, 0, 2, 0, 1, 0, 1, 0, 2, 0, 1, 2, 2, 0, 0, 0, 1, 2, 2, 0, 0, 2, 1, 2], [0, 2, 2, 2, 2, 2, 1, 2, 0, 2, 0, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 0, 1, 0, 1, 1, 1, 2, 1, 2, 2, 2, 0, 1, 2, 1, 2, 1, 1, 2, 2, 2, 0, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2], [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 1, 1, 0, 

In [ ]:
(theta, phi, Z, W) = lda(dataset, alpha = [0.5] * K, beta = [0.6] * V)

/usr/local/lib/python3.7/dist-packages/pymc/MCMC.py:81: UserWarning: Instantiating a Model object directly is deprecated. We recommend passing variables directly to the Model subclass.
  warnings.warn(message)


 [-----------------100%-----------------] 20001 of 20000 complete in 2747.0 sec
The topic of each word in each document:

[2 0 0 2 2 2 0 1 2 2 2 2 2 1 2 0 0 2 2 2 2 2 2 1 1 2 2 0 1 0 2 2 1 2 1 2 2
 1 0 2 0 1 1 2 2 1 1 2 2 0 2 2 0 1 2 2 1 0 0 0 2 1 0 2 0 0 2 0 1 0 2 2 1 0
 1 2 0 0 2 1 2]
[1 2 0 2 1 2 2 0 1 2 0 1 0 1 1 2 2 2 0 1 0 0 0 1 2 2 1 2 2 2 0 1 1 1 2 2 2
 1 1 2 1 0 1 1 0 1 2 1 1 2 2 2 1 1 1 1 0 2 0 2 1 1 2 2 0 1 2 2 0 0]
[0 1 0 1 0 1 0 1 0 1 2 2 1 1 0 2 0 2 2 1 2 1 0 0 2 0 0 0 0 0 2 1 1 0 2 1 0
 0 1 0 0 0 1 0 0 1 0 1 2 0 0 0 0 2 1 1 1 1 1 0 2 2 1 1 2 1 0 1 1 2 1 0 2 2
 2 2 2 0 2 1 2 0 1 0 2 0]
[0 1 2 1 1 2 0 1 2 0 1 1 0 1 0 2 0 0 0 0 1 0 0 2 0 0 1 0 2 1 0 0 1 0 2 0 0
 0 0 0 0 1 0 0 1 0 2 0 0 0 0 1 1 0 0 0 1 0 0 0 2 2 1 2 0 0 0 1 0 2 0 0 1 1
 0 0 0 1 0]
[1 2 2 2 2 2 0 2 1 0 0 2 2 1 1 1 0 0 2 1 2 2 0 2 0 0 2 2 1 0 0 0 2 0 2 0 0
 0 2 0 1 2 0 2 0 1 2 2 2 2 2 1 1 0 0 2 2 2 2 0 0 2 2 1 2 2 0 0 2 2 1 1 0 0
 0 2]
[0 2 2 2 2 2 2 0 2 2 2 2 0 0 0 0 0 1 1 2 0 0 2 2 0 0 0 2 2 0 0 2 1 2 1 1 0


In [ ]:
Z_matrix = []
for i in range(len(Z)):
        Z_matrix.append(list(Z[i].value))
print(Z_matrix)

topic_acc(Z_matrix)

[[2, 0, 0, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 1, 2, 0, 0, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 0, 1, 0, 2, 2, 1, 2, 1, 2, 2, 1, 0, 2, 0, 1, 1, 2, 2, 1, 1, 2, 2, 0, 2, 2, 0, 1, 2, 2, 1, 0, 0, 0, 2, 1, 0, 2, 0, 0, 2, 0, 1, 0, 2, 2, 1, 0, 1, 2, 0, 0, 2, 1, 2], [1, 2, 0, 2, 1, 2, 2, 0, 1, 2, 0, 1, 0, 1, 1, 2, 2, 2, 0, 1, 0, 0, 0, 1, 2, 2, 1, 2, 2, 2, 0, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 0, 1, 1, 0, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 0, 2, 0, 2, 1, 1, 2, 2, 0, 1, 2, 2, 0, 0], [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 2, 2, 1, 1, 0, 2, 0, 2, 2, 1, 2, 1, 0, 0, 2, 0, 0, 0, 0, 0, 2, 1, 1, 0, 2, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 2, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 0, 2, 2, 1, 1, 2, 1, 0, 1, 1, 2, 1, 0, 2, 2, 2, 2, 2, 0, 2, 1, 2, 0, 1, 0, 2, 0], [0, 1, 2, 1, 1, 2, 0, 1, 2, 0, 1, 1, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 1, 0, 2, 1, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 2, 2, 1, 2, 0, 0, 0, 1, 0, 2, 0, 0, 1, 1, 0, 0, 0, 1, 0], [1, 2, 2, 2, 2, 2, 0, 2, 1, 0, 0, 2, 2, 1, 

In [225]:
(theta, phi, Z, W) = lda(dataset, alpha = [0.2] * K, beta = [0.6] * V)

/usr/local/lib/python3.7/dist-packages/pymc/MCMC.py:81: UserWarning: Instantiating a Model object directly is deprecated. We recommend passing variables directly to the Model subclass.
  warnings.warn(message)


 [-----------------100%-----------------] 20000 of 20000 complete in 3112.7 sec
The topic of each word in each document:

[1 2 2 1 1 2 2 1 2 2 1 1 2 0 1 2 2 0 2 2 2 2 2 2 1 1 2 2 2 2 1 1 1 2 2 2 2
 1 1 2 2 1 2 2 1 1 2 2 1 1 2 1 1 1 1 2 1 1 1 2 1 1 1 0 1 2 1 2 1 1 2 2 0 2
 1 1 0 1 1 1 2]
[0 1 2 0 0 1 0 2 0 1 2 2 1 0 1 2 1 2 2 0 0 1 2 1 1 0 2 0 0 1 2 2 0 2 0 0 1
 2 1 1 1 2 2 2 0 2 2 2 0 1 0 0 0 2 0 2 1 0 1 2 1 2 0 0 0 2 0 1 2 1]
[0 0 1 0 2 2 2 1 0 2 2 0 0 0 2 1 0 0 1 1 2 2 1 0 0 1 2 0 0 1 2 2 1 0 1 1 0
 2 2 2 2 1 1 2 2 2 1 1 1 2 0 2 1 0 1 1 1 0 0 2 0 0 0 1 1 2 2 2 0 1 1 2 0 0
 2 0 1 0 0 1 2 1 2 2 1 1]
[2 0 0 2 2 0 2 1 2 0 2 1 0 1 0 1 2 1 0 0 1 0 1 1 0 2 0 0 0 0 2 1 2 2 2 1 0
 2 1 2 0 0 1 1 2 1 1 2 2 2 2 0 0 1 2 2 2 2 2 0 0 1 0 1 2 2 1 1 0 2 0 2 2 0
 0 2 2 0 2]
[0 0 0 2 0 0 0 0 2 0 0 0 0 0 0 2 2 0 0 0 2 0 2 0 0 2 2 0 0 0 0 2 2 2 2 2 0
 0 2 2 2 2 0 2 0 2 2 2 2 0 0 2 2 0 2 0 0 2 2 0 2 0 0 0 0 0 2 0 2 2 0 0 0 0
 2 2]
[0 0 1 1 1 1 1 0 2 2 1 1 0 1 2 1 0 1 1 1 0 1 2 1 2 0 0 1 1 0 1 0 0 1 0 0 0


In [226]:
Z_matrix = []
for i in range(len(Z)):
        Z_matrix.append(list(Z[i].value))
print(Z_matrix)

topic_acc(Z_matrix)

[[1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 0, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 0, 1, 2, 1, 2, 1, 1, 2, 2, 0, 2, 1, 1, 0, 1, 1, 1, 2], [0, 1, 2, 0, 0, 1, 0, 2, 0, 1, 2, 2, 1, 0, 1, 2, 1, 2, 2, 0, 0, 1, 2, 1, 1, 0, 2, 0, 0, 1, 2, 2, 0, 2, 0, 0, 1, 2, 1, 1, 1, 2, 2, 2, 0, 2, 2, 2, 0, 1, 0, 0, 0, 2, 0, 2, 1, 0, 1, 2, 1, 2, 0, 0, 0, 2, 0, 1, 2, 1], [0, 0, 1, 0, 2, 2, 2, 1, 0, 2, 2, 0, 0, 0, 2, 1, 0, 0, 1, 1, 2, 2, 1, 0, 0, 1, 2, 0, 0, 1, 2, 2, 1, 0, 1, 1, 0, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 0, 2, 1, 0, 1, 1, 1, 0, 0, 2, 0, 0, 0, 1, 1, 2, 2, 2, 0, 1, 1, 2, 0, 0, 2, 0, 1, 0, 0, 1, 2, 1, 2, 2, 1, 1], [2, 0, 0, 2, 2, 0, 2, 1, 2, 0, 2, 1, 0, 1, 0, 1, 2, 1, 0, 0, 1, 0, 1, 1, 0, 2, 0, 0, 0, 0, 2, 1, 2, 2, 2, 1, 0, 2, 1, 2, 0, 0, 1, 1, 2, 1, 1, 2, 2, 2, 2, 0, 0, 1, 2, 2, 2, 2, 2, 0, 0, 1, 0, 1, 2, 2, 1, 1, 0, 2, 0, 2, 2, 0, 0, 2, 2, 0, 2], [0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 

In [103]:
(theta, phi, Z, W) = lda(dataset, alpha = [0.2] * K, beta = [1.0] * V)

/usr/local/lib/python3.7/dist-packages/pymc/MCMC.py:81: UserWarning: Instantiating a Model object directly is deprecated. We recommend passing variables directly to the Model subclass.
  warnings.warn(message)


 [-----------------100%-----------------] 20000 of 20000 complete in 2588.5 sec
The topic of each word in each document:

[2 2 2 2 2 0 1 2 0 2 0 2 0 0 2 2 0 2 0 1 0 2 2 2 2 2 2 0 0 2 0 2 2 1 0 2 0
 0 0 0 0 0 0 0 2 1 2 2 1 0 2 0 0 1 0 2 2 2 2 0 0 0 0 2 2 0 2 2 0 1 0 1 0 1
 2 2 0 2 2 0 1]
[2 1 1 1 1 1 2 1 2 1 2 1 1 1 1 2 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1 1 1 1 1 1
 2 2 1 1 1 2 1 1 2 1 2 2 2 1 2 2 2 2 1 2 1 1 1 2 1 1 2 2 1 1 1 1 1]
[2 0 0 0 2 2 2 0 0 0 0 2 0 2 0 2 0 0 0 2 0 0 0 0 0 2 0 2 2 0 0 2 0 2 0 0 0
 2 2 0 0 2 2 2 2 2 2 0 0 0 0 2 0 2 0 2 2 0 0 0 0 0 0 0 0 0 2 0 2 2 2 0 0 0
 0 0 0 0 0 2 2]
[0 0 0 0 2 0 2 0 1 0 1 0 1 0 0 0 0 1 2 0 2 0 0 2 2 0 0 2 1 0 0 2 0 0 2 1 1
 0 0 1 0 2 0 0 0 0 1 2 0 2 0 0 0 0 2 1 0 0 0 1 2 0 0 2 0 0 2 0 0 0 2 0 0 0
 0 0 2 1 0 2 2 2 0 0 1 0]
[1 2 1 2 2 2 1 1 1 2 1 2 1 0 1 0 1 2 1 2 2 1 1 1 2 2 2 0 1 2 2 1 2 2 2 1 0
 1 0 0 1 2 0 2 2 2 1 2 2 2 2 2 2 2 1 2 1 1 2 2 1 2 2 0 2 0 1 2 2 2 1 2 2 1
 2 1 1 1 2]
[0 2 2 2 2 2 1 0 2 0 2 0 0 0 0 2 2 2 0 0 2 2 0 2 0 0 0 0 0 0 2 2 

In [104]:
Z_matrix = []
for i in range(len(Z)):
        Z_matrix.append(list(Z[i].value))
print(Z_matrix)

topic_acc(Z_matrix)

[[2, 2, 2, 2, 2, 0, 1, 2, 0, 2, 0, 2, 0, 0, 2, 2, 0, 2, 0, 1, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 2, 2, 1, 0, 2, 0, 0, 1, 0, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 0, 2, 2, 0, 1, 0, 1, 0, 1, 2, 2, 0, 2, 2, 0, 1], [2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1], [2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2], [0, 0, 0, 0, 2, 0, 2, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 2, 0, 2, 0, 0, 2, 2, 0, 0, 2, 1, 0, 0, 2, 0, 0, 2, 1, 1, 0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 2, 0, 2, 0, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 1, 0, 2, 2, 2, 0, 0, 1, 0], [1, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 

## Conclusions task 1

As we can see from the previous runs, an alpha of 0,2 (small) and a beta of 1,0 (moderate) had the best results. The average accuracy of this model was 36%. The small accuracy can be due to the the relative small size of the documents (only 150 words per document before removing stop words) and also to the small number of iterations (20000, with a burn in period of 1000) which are not enough for convergence. We can see that 50% of the documents have the correct most frequent topic.

The small number of words used for each document means that there are less chances of the same words from the same topic to be repeted between documents, thus the small accuracy.


# Topic-based similarity measures


For this task I implemented a number of similarity measures proposed for LDA in these papers https://www.cs.memphis.edu/~vrus/publications/2013/CICLing-2013.RusNiraulaBanjade.pdf, https://aclanthology.org/E14-4005.pdf.

I have chosen 3 similarity measures based on the distribution over topics (theta), as each document can be viewed as a distribution over topics. Thus, we can compute the similarity between 2 documents, by computing the similarity of their distributions. The 3 measures below return the dissimilarity between 2 distributions, so they will tell us how diffrent two distributions are (0 meaning identical).

- Kullback-Leibler (KL) divergence -> takes two distributions p and q and computes the distance between them

KL(p, q) = sum(pi * log(pi / qi)), where i defines a topic (i in [0, K))

The drawbacks of KL divergence are that, if qi is 0 it is not defined, and is not symmetric.

- Jensen-Shannon divergence -> takes two distributions p and q and computes the distance between them, while solving the asymmetry problem of KL by considering the average of pi and qi

JS(p, q) = 1/2 * KL(p, m) + 1/2 * KL(q, m), where m = 1/2 * (p + q)

- Hellinger distance -> similar to JS distance, it removes the drawback of KL divergence. An advantage of Hellinger distance is that it is very easy to compute the similarity from the distance, by substractig the result from 1.

HD(p, q) = 1/2 * sqrt(sum( sqrt(pi) - sqrt(qi) )^2)

In [92]:
def kl(p, q):
    return np.sum(p * np.log10(p / q))

In [93]:
def js(p, q):
    m = (p + q) / 2
    return kl(p, m) / 2 + kl(q, m) / 2

In [94]:
def hd(p, q):
    return 1./np.sqrt(2) * np.sqrt(np.sum(np.square(np.sqrt(p) - np.sqrt(q))))

In [95]:
print(dataset)

[[0, 1, 2, 3, 4, 5, 1, 6, 7, 8, 3, 9, 10, 11, 12, 13, 1, 14, 15, 16, 17, 13, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 8, 7, 35, 36, 37, 38, 39, 40, 41, 10, 42, 43, 44, 45, 46, 47, 37, 48, 1, 49, 50, 4, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 30, 63, 64, 65, 66, 0, 67, 68], [69, 70, 71, 72, 73, 74, 75, 76, 69, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 69, 87, 62, 31, 88, 89, 62, 90, 91, 92, 93, 94, 95, 96, 71, 97, 98, 84, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 74, 75, 103, 32, 69, 110, 96, 71, 111, 112, 113, 108, 114, 115, 116, 117, 118, 119, 98, 120, 121, 122], [123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 123, 136, 137, 79, 128, 127, 138, 126, 123, 139, 56, 140, 141, 142, 143, 144, 145, 146, 12, 136, 32, 147, 148, 149, 150, 151, 152, 153, 154, 155, 29, 156, 19, 16, 157, 158, 159, 160, 132, 46, 161, 162, 163, 164, 165, 133, 166, 167, 79, 126, 168, 131, 29, 169, 170, 171, 172, 158, 79, 132, 118, 173, 174, 175, 176, 123,

In [96]:
#combine 2 documents to show that the similarity measures work properly
new_doc1 = dataset[0][:50] + dataset[-1][50:]
new_doc2 = dataset[-1][:50] + dataset[0][50:]
print(new_doc1)
print(new_doc2)

[0, 1, 2, 3, 4, 5, 1, 6, 7, 8, 3, 9, 10, 11, 12, 13, 1, 14, 15, 16, 17, 13, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 8, 7, 35, 36, 37, 38, 39, 40, 41, 10, 42, 164, 1068, 1267, 1257, 1258, 1268, 1249, 1269, 1247, 1263, 1270, 1271, 991, 1262, 983, 164, 1068, 1272, 29, 1273, 1247, 1258, 1249, 65, 1248, 897, 476, 1274, 1275]
[1246, 1247, 1248, 1068, 29, 353, 1247, 1249, 1068, 1250, 58, 274, 1248, 609, 1251, 435, 1252, 1253, 1254, 1255, 1247, 1249, 1256, 121, 1068, 1257, 1258, 1256, 1259, 1260, 1249, 967, 1261, 1247, 1108, 991, 1262, 1263, 898, 1264, 1265, 461, 864, 73, 117, 1248, 1249, 955, 1266, 1263, 43, 44, 45, 46, 47, 37, 48, 1, 49, 50, 4, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 30, 63, 64, 65, 66, 0, 67, 68]


In [97]:
dataset[0] = new_doc2
dataset[-1] = new_doc1
print(dataset)

[[1246, 1247, 1248, 1068, 29, 353, 1247, 1249, 1068, 1250, 58, 274, 1248, 609, 1251, 435, 1252, 1253, 1254, 1255, 1247, 1249, 1256, 121, 1068, 1257, 1258, 1256, 1259, 1260, 1249, 967, 1261, 1247, 1108, 991, 1262, 1263, 898, 1264, 1265, 461, 864, 73, 117, 1248, 1249, 955, 1266, 1263, 43, 44, 45, 46, 47, 37, 48, 1, 49, 50, 4, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 30, 63, 64, 65, 66, 0, 67, 68], [69, 70, 71, 72, 73, 74, 75, 76, 69, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 69, 87, 62, 31, 88, 89, 62, 90, 91, 92, 93, 94, 95, 96, 71, 97, 98, 84, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 74, 75, 103, 32, 69, 110, 96, 71, 111, 112, 113, 108, 114, 115, 116, 117, 118, 119, 98, 120, 121, 122], [123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 123, 136, 137, 79, 128, 127, 138, 126, 123, 139, 56, 140, 141, 142, 143, 144, 145, 146, 12, 136, 32, 147, 148, 149, 150, 151, 152, 153, 154, 155, 29, 156, 19, 16, 157, 158, 159, 160, 132, 46, 161, 162, 163, 164, 165, 133,

In [98]:
(theta, phi, Z, W) = lda(dataset, alpha=[0.2] * K, beta=[0.6] * V)

/usr/local/lib/python3.7/dist-packages/pymc/MCMC.py:81: UserWarning: Instantiating a Model object directly is deprecated. We recommend passing variables directly to the Model subclass.
  warnings.warn(message)


 [-----------------100%-----------------] 20000 of 20000 complete in 3196.7 sec
The topic of each word in each document:

[0 2 2 2 0 1 0 1 0 1 2 1 2 1 1 1 1 0 2 1 2 1 1 2 2 0 2 0 0 2 1 0 1 2 2 0 2
 0 1 0 1 2 1 1 0 2 1 1 2 2 0 2 0 0 0 0 1 2 1 0 1 0 1 1 1 1 2 2 0 0 0 1 0 1
 2 1 1 2 1 0 1]
[2 2 2 2 2 0 0 0 0 0 2 0 0 2 2 2 0 0 2 0 0 0 2 2 2 0 0 2 2 0 0 2 2 2 0 2 0
 0 0 2 0 0 0 2 2 0 2 2 2 0 2 2 0 0 2 2 2 0 2 0 0 2 0 0 2 0 2 0 2 2]
[2 1 0 0 0 2 0 2 1 2 0 0 2 1 1 0 0 0 0 1 0 2 1 2 2 0 1 2 2 0 0 0 0 2 0 0 2
 2 0 1 1 0 0 0 2 2 0 0 1 0 2 0 2 2 1 2 2 1 1 2 2 2 0 2 1 0 0 2 0 1 1 0 2 0
 0 0 0 2 1 2 2]
[2 2 2 2 1 2 2 1 2 2 2 1 2 2 2 2 1 2 2 2 2 2 2 1 1 2 2 2 2 2 1 2 2 2 2 2 1
 2 2 1 2 2 2 2 2 2 2 1 1 2 2 1 2 2 2 1 1 2 1 1 2 2 2 2 2 2 1 2 2 2 2 2 2 1
 2 1 2 1 2 2 2 2 2 1 1 1]
[2 2 2 0 1 1 2 2 2 2 1 0 1 2 0 1 1 1 1 0 0 0 2 2 1 2 1 1 0 2 2 1 2 2 1 2 1
 1 1 2 1 2 1 2 1 2 2 2 2 1 2 2 2 1 0 1 0 2 1 1 1 2 1 2 2 2 1 2 1 2 0 2 1 2
 2 1 0 1 2]
[1 2 1 1 1 2 2 1 1 1 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1 1 

In [99]:
Z_matrix = []
for i in range(len(Z)):
        Z_matrix.append(list(Z[i].value))
print(Z_matrix)

topic_acc(Z_matrix)

[[0, 2, 2, 2, 0, 1, 0, 1, 0, 1, 2, 1, 2, 1, 1, 1, 1, 0, 2, 1, 2, 1, 1, 2, 2, 0, 2, 0, 0, 2, 1, 0, 1, 2, 2, 0, 2, 0, 1, 0, 1, 2, 1, 1, 0, 2, 1, 1, 2, 2, 0, 2, 0, 0, 0, 0, 1, 2, 1, 0, 1, 0, 1, 1, 1, 1, 2, 2, 0, 0, 0, 1, 0, 1, 2, 1, 1, 2, 1, 0, 1], [2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, 0, 2, 2, 2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 2], [2, 1, 0, 0, 0, 2, 0, 2, 1, 2, 0, 0, 2, 1, 1, 0, 0, 0, 0, 1, 0, 2, 1, 2, 2, 0, 1, 2, 2, 0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 1, 1, 0, 0, 0, 2, 2, 0, 0, 1, 0, 2, 0, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 0, 2, 1, 0, 0, 2, 0, 1, 1, 0, 2, 0, 0, 0, 0, 2, 1, 2, 2], [2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1], [2, 2, 2, 0, 1, 1, 2, 2, 2, 2, 1, 0, 

In [100]:
print("Jensen-Shannon divergence \n")
for i in range(len(theta)):
    for j in range(len(theta)):
        print("Documents", i, j, " have the following divergence: ", js(theta[i].value, theta[j].value))

Jensen-Shannon divergence 

Documents 0 0  have the following divergence:  0.0
Documents 0 1  have the following divergence:  0.07760384122469294
Documents 0 2  have the following divergence:  0.006539466638256145
Documents 0 3  have the following divergence:  0.07462680726875413
Documents 0 4  have the following divergence:  0.007296443931027154
Documents 0 5  have the following divergence:  0.06397369661874525
Documents 0 6  have the following divergence:  0.057860631115537395
Documents 0 7  have the following divergence:  0.027492855250657155
Documents 0 8  have the following divergence:  0.002197132958695034
Documents 0 9  have the following divergence:  0.015591564523167786
Documents 0 10  have the following divergence:  0.002891052542391877
Documents 0 11  have the following divergence:  0.03305732306882997
Documents 0 12  have the following divergence:  0.0014020104948474091
Documents 0 13  have the following divergence:  0.021741006449137533
Documents 0 14  have the following d

In [101]:
print("Kullback-Leibler divergence\n")
for i in range(len(theta)):
    for j in range(len(theta)):
        print("Documents", i, j, " have the following divergence: ", kl(theta[i].value, theta[j].value))

Kullback-Leibler divergence

Documents 0 0  have the following divergence:  0.0
Documents 0 1  have the following divergence:  3.1946875605116576
Documents 0 2  have the following divergence:  0.026951760697650325
Documents 0 3  have the following divergence:  0.7101017960738579
Documents 0 4  have the following divergence:  0.031142084166020205
Documents 0 5  have the following divergence:  0.6644398491911065
Documents 0 6  have the following divergence:  0.32562207029291856
Documents 0 7  have the following divergence:  0.12356698277511556
Documents 0 8  have the following divergence:  0.008778162897072093
Documents 0 9  have the following divergence:  0.06582624503278903
Documents 0 10  have the following divergence:  0.011753763791457303
Documents 0 11  have the following divergence:  0.13511878255796056
Documents 0 12  have the following divergence:  0.005833299273589124
Documents 0 13  have the following divergence:  0.10029784241493878
Documents 0 14  have the following divergen

In [102]:
print("Hellinger divergence\n")
for i in range(len(theta)):
    for j in range(len(theta)):
        print("Documents", i, j, " have the following divergence: ", hd(theta[i].value, theta[j].value), "\t and the following similarity: ", 1 - hd(theta[i].value, theta[j].value))

Hellinger divergence

Documents 0 0  have the following divergence:  0.0 	 and the following similarity:  1.0
Documents 0 1  have the following divergence:  0.49314039777420393 	 and the following similarity:  0.5068596022257961
Documents 0 2  have the following divergence:  0.1229424003129564 	 and the following similarity:  0.8770575996870436
Documents 0 3  have the following divergence:  0.4535812744012038 	 and the following similarity:  0.5464187255987962
Documents 0 4  have the following divergence:  0.13001210728976278 	 and the following similarity:  0.8699878927102372
Documents 0 5  have the following divergence:  0.4239978082993803 	 and the following similarity:  0.5760021917006197
Documents 0 6  have the following divergence:  0.3777810896561157 	 and the following similarity:  0.6222189103438843
Documents 0 7  have the following divergence:  0.2542218912098416 	 and the following similarity:  0.7457781087901584
Documents 0 8  have the following divergence:  0.0711730520665

We can observe that the similarity between the 2 combined documents (the first and the last one) have a high value and the dissimilarity between them is, by contrast, low.

# Assigning a topic to a new document

In this section I will present a mathematical approach in which we can estimate the probability to have a topic t, given the fact that we have a new document d, noted p(t/d).

t = topic

d = document

di = word i in document

Followin the bayes theorem, we have that:
p(t/d) = p(d/t) * p(t) / p(d)

At the same time, consdering that the words are independent variables, we can state that p(d/t) = p(d1...dn/t) = prod(p(di, t))

The above p(di/t) is actually the word distribution for each topic (phi).

p(t) is simply the probability of a topic, which is 1/K.

p(d) is the probability of a document, which is 1/(M + 1).

Having all the necessary information, it is now fairly simple to compute the desired probability.

For computational purpose, I have applied log to the probability and added a small value (epsilon) to each value in phi.

log(p(t/d)) = log(p(d/t) + eps) + log(p(t)) - log(p(d))

log(p(d/t) + eps) = sum(log(p(di/t) + eps))

In [105]:
for i in range(len(phi)):
    print("Topic", i, ": ", phi[i].value)

Topic 0 :  [[4.20770338e-05 7.79848126e-04 1.34980001e-03 ... 2.14564969e-04
  2.28955877e-03 2.20689019e-04]]
Topic 1 :  [[1.74255472e-03 5.60003421e-04 3.08520380e-03 ... 4.79002328e-04
  1.50321814e-05 2.89119578e-03]]
Topic 2 :  [[0.00314537 0.00213355 0.00165881 ... 0.0001673  0.00010966 0.00032949]]


In [106]:
new_d = []
for w in new_f[0]:
    if w in index_to_word:
      new_d.append(w)

print(new_d)

[804, 147, 544, 164, 804, 536, 267, 796, 652, 473, 677, 461, 1191, 804, 553, 473, 804, 752, 237, 695, 246, 473, 478, 62, 807, 844, 267, 222, 473]


In [107]:
eps = 1e-5
def compute_prod_prob(phi, doc, topic):
    prob = 0
    for di in doc:
        prob += log(list(phi[topic].value)[0][di] + eps)
    return prob

In [108]:
def compute_prob(p_d_t, p_d, p_t, topic):
    return p_d_t + log(p_t) - log(p_d)

In [109]:
p_t = 1/K
p_d = 1/(M+1)
probs = []
for topic in range(K):
    p_d_t = compute_prod_prob(phi, new_d, topic)
    p_t_d = compute_prob(p_d_t, p_d, p_t, topic)
    probs.append(exp(p_t_d))
    print("Probability for topic ", topic, "is", exp(p_t_d))

max_prob = max(probs)
max_topic = probs.index(max_prob)

print("The topic with maximum probability is ", max_topic)

Probability for topic  0 is 4.3982976130066025e-97
Probability for topic  1 is 4.1986083162981516e-94
Probability for topic  2 is 3.997912093629488e-93
The topic with maximum probability is  2


The topic should have been 1, which is the second most probable.